In [1]:
# !pip install split-folders

In [2]:
# import splitfolders
# splitfolders.ratio('DATASET_2', output="output", seed=1337, ratio=(.7, 0.2,0.1))

Copying files: 2339 files [00:05, 428.03 files/s]


In [5]:
# import splitfolders 

# #### input dataset that want to split
# input_folder = 'DATASET_2'  

# output_folder= 'Splitted_Dataset_2'

# splitfolders.ratio(input_folder, output= output_folder, seed=1337, ratio = (0.7, 0.2, 0.1))

Copying files: 2339 files [00:04, 522.85 files/s]


In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras
from tensorflow.keras import backend as k
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import os
import glob
import numpy as np
import json

with open('model.json', 'r') as json_file:
        json_savedModel= json_file.read()

    
model = tf.keras.models.model_from_json(json_savedModel)
model.load_weights('model.h5')

In [15]:
def get_datagen():    
    datagen = ImageDataGenerator(
        #rescale=1./255,
        rotation_range=16,
        width_shift_range=0.1,
        height_shift_range=0.1,
#         horizontal_flip=True,
        shear_range=0.2,
        zoom_range=0.1
    )
    test_datagen = ImageDataGenerator(
        #rescale=1./255,
    )
    return datagen,test_datagen


def get_data(train_datagen,test_datagen):
    train_it = train_datagen.flow_from_directory('Splitted_Dataset_2/train', class_mode='categorical',target_size=(224, 224))

    test_it = test_datagen.flow_from_directory('Splitted_Dataset_2/val', class_mode='categorical',target_size=(224, 224))
    return train_it,test_it

In [20]:
def get_model(vgg): 
    
    x = tf.keras.layers.Flatten()(vgg.output)
    # three hidden layers
    x = tf.keras.layers.Dense(256, activation='selu')(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    # final softmax layer with two categories (dog and cat)
    predictions = tf.keras.layers.Dense(52, activation='softmax')(x)
    model = tf.keras.Model(inputs=vgg.input,outputs=predictions)
    model.summary()
    return model


In [19]:
def get_Resnet():
    from tensorflow.keras.applications import ResNet50
    vgg = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3),pooling='avg')
    for i in range(0,len(vgg.layers)-20):
        vgg.layers[i].trainable = False
    vgg.summary()
    return vgg

In [16]:
datagen,test_datagen=get_datagen()
train_it,test_it=get_data(datagen,test_datagen)

Found 1610 images belonging to 52 classes.
Found 465 images belonging to 52 classes.


In [17]:
def train(ep):
    model = get_Resnet()
    model=get_model(model)
    model.compile(loss='categorical_crossentropy',
                      optimizer=tf.keras.optimizers.Adamax(),
                      metrics=['acc'])
    history = model.fit(train_it, epochs = ep,steps_per_epoch=16, validation_data=test_it, validation_steps=8)
    return model,history

In [21]:
train(25)

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

...

                                                                                                  
 conv3_block4_1_conv (Conv2D)   (None, 28, 28, 128)  65664       ['conv3_block3_out[0][0]']       
                                                                                                  
 conv3_block4_1_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block4_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block4_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block4_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block4_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block4_1_relu[0][0]']    
          

 conv4_block2_add (Add)         (None, 14, 14, 1024  0           ['conv4_block1_out[0][0]',       
                                )                                 'conv4_block2_3_bn[0][0]']      
                                                                                                  
 conv4_block2_out (Activation)  (None, 14, 14, 1024  0           ['conv4_block2_add[0][0]']       
                                )                                                                 
                                                                                                  
 conv4_block3_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block2_out[0][0]']       
                                                                                                  
 conv4_block3_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block3_1_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 conv4_block5_add (Add)         (None, 14, 14, 1024  0           ['conv4_block4_out[0][0]',       
                                )                                 'conv4_block5_3_bn[0][0]']      
                                                                                                  
 conv4_block5_out (Activation)  (None, 14, 14, 1024  0           ['conv4_block5_add[0][0]']       
                                )                                                                 
                                                                                                  
 conv4_block6_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block5_out[0][0]']       
                                                                                                  
 conv4_block6_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block6_1_conv[0][0]']    
 ization) 

                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 7, 7, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 7, 7, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_block2_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block2_add[0][0]']       
                                                                                                  
 conv5_block3_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block2_out[0][0]']       
          

(<keras.engine.functional.Functional at 0x18b41104d30>,
 <keras.callbacks.History at 0x18b3972cf10>)

In [23]:
def save_model(model):
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model.h5")
save_model(model)

In [26]:
plt.figure(figsize=(15,5))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.ylim(0.1,1)
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

<Figure size 1080x360 with 0 Axes>